In [ ]:
import json
from pathlib import Path

import pandas as pd

In [6]:
root_dir = Path().resolve().parent
cache_dir = root_dir / ".cache" / "test"

In [34]:
with open(cache_dir / "out.json", "r") as f:
    output = json.load(f)


scene_data = output["data"]["scene_data"]["scenes"]
image_data = output["data"]["thumbnail_data"]

records = []
for i in range(len(image_data)):
    i_image_data = image_data[i]
    images = i_image_data["images"]
    not_images = {k: v for k, v in i_image_data.items() if k != "images"}

    for image in images:
        record = not_images.copy()
        record.update(image)
        records.append(record)

In [36]:
df = pd.DataFrame(records)
df.head()

,process_index,query,id,snippet,link,thumbnailLink,width,height,rank
0,0,横浜駅 海と観覧車 夜景 住宅街,img_1,横浜在住歴20年、おしゃれと港だけじゃない。横浜の住みやすさとエリア ...,https://www.housecom.jp/kurashiate/images/2022...,https://encrypted-tbn0.gstatic.com/images?q=tb...,1999,1333,1
1,0,横浜駅 海と観覧車 夜景 住宅街,img_2,横浜の夜景スポット26選！デートや撮影におすすめの場所がわかる,https://nightscape.tokyo/wp-content/uploads/20...,https://encrypted-tbn0.gstatic.com/images?q=tb...,1300,866,2
2,0,横浜駅 海と観覧車 夜景 住宅街,img_3,横浜で夜景がきれいに見えるスポット28選 | TABIPPO.NET,https://tabippo.net/wp-content/uploads/2017082...,https://encrypted-tbn0.gstatic.com/images?q=tb...,680,453,3
3,0,横浜駅 海と観覧車 夜景 住宅街,img_4,神奈川県の夜景スポット記事の一覧 | 夜景FANマガジン,https://yakei-fan.com/images/magazine/yakei/pi...,https://encrypted-tbn0.gstatic.com/images?q=tb...,1200,800,4
4,0,横浜駅 海と観覧車 夜景 住宅街,img_5,横浜ベイホテル東急客室から見るみなとみらいの夜景】 タイムラプスで ...,https://lookaside.instagram.com/seo/google_wid...,https://encrypted-tbn0.gstatic.com/images?q=tb...,720,720,5


In [ ]:
df

In [40]:
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe

In [38]:
sa_filepath = root_dir / "sns_ai_automation_agency/sa-key.json"

In [41]:
# 認証情報の設定
scopes = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]

# サービスアカウントキーのパスを指定
creds = Credentials.from_service_account_file(sa_filepath, scopes=scopes)

# gspreadクライアントの初期化
client = gspread.authorize(creds)

In [46]:
sheet_url = "https://docs.google.com/spreadsheets/d/1bzROAf4BHIdq551QpWbRL1xRnB2hFqJGQ9Mu9Vj0w-c/edit?usp=sharing"

In [47]:
# スプレッドシートを開く
sheet = client.open_by_url(sheet_url)

# ワークシートを取得(最初のシート)
worksheet = sheet.get_worksheet(0)

# データフレームをアップロード
set_with_dataframe(worksheet, df)